In [1]:
import pandas as pd
import psycopg2 
from textblob import TextBlob as tb

In [2]:
#connecting to postgres from SDSC servers, MAKE SURE TO VPN
conn_string = "host='132.249.238.27' dbname='bookstore_dp' user='student' password='123456'"
print ("Connecting to database\n	->%s"% (conn_string))
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print ("Connected!\n")

Connecting to database
	->host='132.249.238.27' dbname='bookstore_dp' user='student' password='123456'
Connected!



In [21]:
sql1="SELECT * from reviews limit 1000"
cursor.execute(sql1)
records=cursor.fetchall()
#Find Column Names for dataframe headers
colnames = [desc[0] for desc in cursor.description]
#constructing dataframe
df_reviews=pd.DataFrame(records, columns=colnames)

In [22]:
df_reviews.head()

,reviewid,asin,reviewername,helpful,unixreviewtime,reviewtext,overall,reviewtime,summary
0,AX9515ESJDLU4,0007267622,"Deann Jackson ""readalicious""","[0, 0]",1265500800,I have been looking forward to this book ever ...,3.00,2010-02-07,"Readable, I guess."
1,A3O5Z02LCF1H8G,0007267622,Debbie,"[1, 2]",1300320000,"I have never read a Dean Koontz book before, b...",1.00,2011-03-17,Waste of my time
2,A3AOOJOCST7YIB,0007267622,deborah,"[5, 8]",1259193600,i dont know how he does it but dean koontz ha...,5.00,2009-11-26,breathless
3,A3R9X003XW0LNR,0007267622,"Deborah Verlen ""Deborah""","[1, 2]",1261699200,I am a huge Dean Koontz fan and also absolutel...,3.00,2009-12-25,I'm not sure what to say...
4,A1LR05CIW9Z65M,0007267622,"Debs ""maplemaiden""","[2, 6]",1259539200,I have been a Kuntz fan for years and this one...,5.00,2009-11-30,Breathless


In [23]:
df_reviews.shape

(1000, 9)

In [24]:
# calculating sentiment polarity, the values ranges from -1 to 1
polarity_measure=[]
for i in range(df_reviews.shape[0]):
    str1 = str(df_reviews.reviewtext[i])
    blob=tb(str1)
    polarity_measure.append(blob.sentiment.polarity)

In [25]:
se = pd.Series(polarity_measure)
df_reviews['Sentiment_polarity'] = se.values


In [26]:
df_reviews.head()

,reviewid,asin,reviewername,helpful,unixreviewtime,reviewtext,overall,reviewtime,summary,Sentiment_polarity
0,AX9515ESJDLU4,0007267622,"Deann Jackson ""readalicious""","[0, 0]",1265500800,I have been looking forward to this book ever ...,3.00,2010-02-07,"Readable, I guess.",-0.005833
1,A3O5Z02LCF1H8G,0007267622,Debbie,"[1, 2]",1300320000,"I have never read a Dean Koontz book before, b...",1.00,2011-03-17,Waste of my time,0.165774
2,A3AOOJOCST7YIB,0007267622,deborah,"[5, 8]",1259193600,i dont know how he does it but dean koontz ha...,5.00,2009-11-26,breathless,0.000000
3,A3R9X003XW0LNR,0007267622,"Deborah Verlen ""Deborah""","[1, 2]",1261699200,I am a huge Dean Koontz fan and also absolutel...,3.00,2009-12-25,I'm not sure what to say...,0.207947
4,A1LR05CIW9Z65M,0007267622,"Debs ""maplemaiden""","[2, 6]",1259539200,I have been a Kuntz fan for years and this one...,5.00,2009-11-30,Breathless,0.550000


In [27]:
#average sentiment polarity per product
df_sentiment=df_reviews.groupby(['asin'],as_index=False)['Sentiment_polarity'].mean()

In [29]:
df_sentiment.head()

,asin,Sentiment_polarity
0,0007267622,0.136762
1,0007267770,0.433333
2,0007267886,0.666667
3,0007268440,0.089849
4,0007268505,0.376935
